## News Feed

In [1]:
import abc
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import json
import pandas as pd
import requests

# import requests
# import urllib.request
# import pandas as pd
# import json
# from pandas.io.json import json_normalize
# import time
# from datetime import date
# from bs4 import BeautifulSoup
# import csv
# from datetime import datetime
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
url = "https://raw.githubusercontent.com/mariko-sawada/FinRL_with_fundamental_data/main/dow_30_fundamental_wrds.csv"
df = pd.read_csv(url)
df.to_csv('dow_30_fundamental.csv', index=False)
df.head()

C:\Users\aminh\AppData\Local\Temp\ipykernel_14160\1260268395.py:2: DtypeWarning: Columns (16,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,dvpsxq,mkvaltq,prccq,prchq,prclq,adjex,ggroup,gind,gsector,gsubind
0,1447,19990630,1999,2,12,INDL,C,D,STD,AXP,...,0.225,NaN,130.1250,142.6250,114.5000,3.0,4020,402020,40,40202010
1,1447,19990930,1999,3,12,INDL,C,D,STD,AXP,...,0.000,NaN,135.0000,150.6250,121.8750,3.0,4020,402020,40,40202010
2,1447,19991231,1999,4,12,INDL,C,D,STD,AXP,...,0.225,NaN,166.2500,168.8750,130.2500,3.0,4020,402020,40,40202010
3,1447,20000331,2000,1,12,INDL,C,D,STD,AXP,...,0.225,NaN,148.9375,169.5000,119.5000,3.0,4020,402020,40,40202010
4,1447,20000630,2000,2,12,INDL,C,D,STD,AXP,...,0.080,NaN,52.1250,57.1875,43.9375,1.0,4020,402020,40,40202010


In [17]:
# class NewsSourceInterface(abc.ABC):
#     ticker = None

#     def get_ticker(self) -> str:
#         return self.ticker

#     @abc.abstractmethod
#     def get_news(self) -> pd.DataFrame:
#         raise NotImplementedError("Needs to be implemented by subclass.")

### EOD Historical Data API

In [10]:
class FundamentalSource():
    API_TOKEN = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"

    def __init__(self, ticker, date_from=None, date_to=None):
        self.ticker = ticker
        self.date_from = date_from
        self.date_to = date_to

    def get_raw_data(self):
        url = "https://eodhistoricaldata.com/api/fundamentals/{}?api_token={}" \
            .format(self.ticker, self.API_TOKEN)
        print(url)
        response = requests.get(url)
        content = response.content
        try:
            parsed = json.loads(content)
        except json.decoder.JSONDecodeError:
            parsed = {}
        return parsed

    @staticmethod
    def to_dataframe(parsed):
        df = pd.DataFrame.from_dict(parsed)
        return df

    def get_news(self):
        parsed = self.get_raw_data()
        # parsed = self.to_dataframe(parsed)
        return parsed

In [11]:
# news_collector = NewsSource("AAPL", num_rows=10)
news_collector = FundamentalSource(
    ticker = "AAPL",
    date_from = '2021-01-01',
    date_to = '2023-06-01'
)
res = news_collector.get_news()

https://eodhistoricaldata.com/api/fundamentals/AAPL?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX


In [12]:
res

{'General': {'Code': 'AAPL',
  'Type': 'Common Stock',
  'Name': 'Apple Inc',
  'Exchange': 'NASDAQ',
  'CurrencyCode': 'USD',
  'CurrencyName': 'US Dollar',
  'CurrencySymbol': '$',
  'CountryName': 'USA',
  'CountryISO': 'US',
  'OpenFigi': 'BBG000B9XRY4',
  'ISIN': 'US0378331005',
  'LEI': 'HWUPKR0MPOU8FGXBT394',
  'PrimaryTicker': 'AAPL.US',
  'CUSIP': '037833100',
  'CIK': '320193',
  'EmployerIdNumber': '94-2404110',
  'FiscalYearEnd': 'September',
  'IPODate': '1980-12-12',
  'InternationalDomestic': 'International/Domestic',
  'Sector': 'Technology',
  'Industry': 'Consumer Electronics',
  'GicSector': 'Information Technology',
  'GicGroup': 'Technology Hardware & Equipment',
  'GicIndustry': 'Technology Hardware, Storage & Peripherals',
  'GicSubIndustry': 'Technology Hardware, Storage & Peripherals',
  'HomeCategory': 'Domestic',
  'IsDelisted': False,
  'Description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and acce

In [59]:
class NewsSource():
    API_TOKEN = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"

    def __init__(self, ticker, num_rows=10, date_from=None, date_to=None):
        self.ticker = ticker
        self.num_rows = num_rows
        self.date_from = date_from
        self.date_to = date_to

    def get_raw_data(self):
        if self.date_from is None:
            url = "https://eodhistoricaldata.com/api/news?api_token={}&s={}&offset=0&limit={}" \
                .format(self.API_TOKEN, self.ticker, self.num_rows)
        else:
            url = "https://eodhistoricaldata.com/api/news?api_token={}&s={}&from={}&to={}&offset=0&limit={}". \
                format(self.API_TOKEN, self.ticker, self.date_from, self.date_to, self.num_rows)
        print(url)
        response = requests.get(url)
        content = response.content
        try:
            parsed = json.loads(content)
        except json.decoder.JSONDecodeError:
            parsed = {}
        return parsed

    @staticmethod
    def to_dataframe(parsed):
        d = {"date": [], "title": [], "content": [], "symbols": [], "tags": [], "sentiment": []}
        for i in range(len(parsed)):
            d["date"].append(parsed[i]["date"])
            d["title"].append(parsed[i]["title"])
            d["content"].append(parsed[i]["content"])
            d["symbols"].append(parsed[i]["symbols"])
            d["tags"].append(parsed[i]["tags"])
            d["sentiment"].append(parsed[i]["sentiment"])
        df = pd.DataFrame.from_dict(d)
        return df

    def get_news(self):
        parsed = self.get_raw_data()
        df = self.to_dataframe(parsed)
        df.drop(["tags"], axis=1, inplace=True)
        return df

In [92]:
# news_collector = NewsSource("AAPL", num_rows=10)
news_collector = NewsSource(
    ticker = "AAPL",
    num_rows = 20,
    date_from = '2021-01-01',
    date_to = '2023-06-01'
)
res = news_collector.get_news()

https://eodhistoricaldata.com/api/news?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX&s=AAPL&from=2021-01-01&to=2023-06-01&offset=0&limit=20


In [93]:
res.head(100)

,date,title,content,symbols,sentiment
0,2023-06-01T22:45:24+00:00,Apple denies hacking thousands of iPhones in R...,Apple\n\nApple has denied allegations that it ...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA]","{'polarity': 0.94, 'neg': 0.055, 'neu': 0.848,..."
1,2023-06-01T22:02:25+00:00,Russia Accuses US Intelligence of Hacking Thou...,(Bloomberg) -- Russia’s main security service ...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA]","{'polarity': -0.942, 'neg': 0.091, 'neu': 0.83..."
2,2023-06-01T21:43:00+00:00,Broadcom CEO Sees Rising AI Chip Demand. Earni...,Chip maker Broadcom reported better-than-expec...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA...","{'polarity': 0.273, 'neg': 0, 'neu': 0.9, 'pos..."
3,2023-06-01T20:55:29+00:00,Dow Jones Today: Debt Vote Drives Index Higher,The Dow Jones Industrial Average moved higher ...,"[AAPL.MX, AAPL.US, AAPL34.SA, AEC1.F, AEC1.XET...","{'polarity': 0.572, 'neg': 0, 'neu': 0.893, 'p..."
4,2023-06-01T20:24:46+00:00,Trillion-dollar companies: How wealthy are the...,"Alphabet, Apple, Microsoft, and Amazon are jus...","[AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F, ABEA.XET...","{'polarity': 0.44, 'neg': 0.061, 'neu': 0.847,..."
5,2023-06-01T19:54:00+00:00,Apple Stock Is Near a Record High. What Could ...,Investors in the iPhone maker will be watching...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA]","{'polarity': 0.494, 'neg': 0, 'neu': 0.862, 'p..."
6,2023-06-01T19:30:32+00:00,Stock market gains powered by just a few stocks,"Apple, Nvidia, Alphabet, Microsoft, and Amazon...","[AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F, ABEA.XET...","{'polarity': 0.649, 'neg': 0, 'neu': 0.826, 'p..."
7,2023-06-01T19:01:00+00:00,Invest Like Warren Buffett With These 3 Stocks,"Warren Buffett, commonly known as the Oracle o...","[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA...","{'polarity': 0.997, 'neg': 0.022, 'neu': 0.836..."
8,2023-06-01T18:35:25+00:00,Why Meta Platforms Stock Was Up Today,Shares of Meta Platforms (NASDAQ: META) were m...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA...","{'polarity': 0.586, 'neg': 0, 'neu': 0.936, 'p..."
9,2023-06-01T18:21:04+00:00,Apple’s Retail Plans Will Push Company Deeper ...,(Bloomberg) -- Apple Inc. is working on plans ...,"[AAPL.MX, AAPL.US, AAPL34.SA, APC.F, APC.XETRA]","{'polarity': 0.995, 'neg': 0.027, 'neu': 0.911..."


In [ ]:
res.to_csv('../datasets/test.csv', index=False)

### RSS

In [ ]:
import queue
import feedparser
from typing import Optional, Dict

q = queue.Queue()  # queue of processed items ready to be ingested by data pipeline
rss_sources = []  # list of rss sources with their own processing function


def process_rss_feed_entry(entry: feedparser.util.FeedParserDict) -> Dict[str, str]:
    return {"title": entry.title, "summary": entry.summary}


class RssSource:
    def __init__(self, url, process_entry=None):
        self.url = url
        self.process_entry = process_entry
        self._newest_id = None

    def get_newest_entry(self):
        feed = feedparser.parse(self.url)
        try:
            entry = feed['entries'][0]
            if self._newest_id is None or entry['id'] != self._newest_id:
                self._newest_id = entry['id']
                return self.process_entry(entry)
        except:
            pass
        return None


def wait_for_newest_rss_feed_entry():
    i = 0
    while True:
        newest_rss_item = rss_sources[i].get_newest_entry()
        if newest_rss_item:
            q.put(newest_rss_item)

        i = (i + 1) % len(rss_sources)


urls = [
    'https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml',
    'https://www.economist.com/united-states/rss.xml',
    'https://www.economist.com/the-americas/rss.xml'
]

for url in urls:
    rss_sources.append(RssSource(url, process_rss_feed_entry))
wait_for_newest_rss_feed_entry()

## Bert Sentiment Analysis

In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:
# load the BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [57]:
# preprocess the text
text = "This is a wonderful movie"
input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])

# classify the text
with torch.no_grad():
    preds = model(input_ids)
preds

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.7822, -0.3086]]), hidden_states=None, attentions=None)

In [54]:
# map the label index to a label
label_idx = preds.logits.argmax().item()
label_map = {0: "negative", 1: "positive"}
sentiment = label_map[label_idx]
print(sentiment)

negative
